In [ ]:
# Install required libraries
!pip install datasets transformers huggingface_hub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.9 MB/s eta 0:00:00


In [ ]:
# Import key libraries and packages
import numpy as np
import os
import pandas as pd

from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TrainingArguments, Trainer

In [ ]:
# Login to HF hub
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
# Disable Weights & Biases
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Load the datasets
train_df = pd.read_csv("/content/drive/MyDrive/waid/Azubi Africa BAP/LP5 - NLP/zindi_challenge/data/Train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/waid/Azubi Africa BAP/LP5 - NLP/zindi_challenge/data/Test.csv")

In [ ]:
# View the training data info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [ ]:
# Drop the rows with nulls from the training data
train_df.dropna(inplace = True)

In [ ]:
# Distribution of tweet sentiments
train_df["label"].value_counts()

 0.0    4908
 1.0    4053
-1.0    1038
Name: label, dtype: int64

## Fine-tuning the DistilBERT model


In [ ]:
# Split the train data into train, eval
train, eval = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

In [ ]:
# Instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", num_labels=3)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Save split data subsets
train.to_csv("/content/drive/MyDrive/waid/Azubi Africa BAP/LP5 - NLP/zindi_challenge/data/training_subset.csv", index=False)
eval.to_csv("/content/drive/MyDrive/waid/Azubi Africa BAP/LP5 - NLP/zindi_challenge/data/eval_subset.csv", index=False)

In [ ]:
# Load the subsetted data
data = load_dataset("csv", 
                    data_files={"train": "/content/drive/MyDrive/waid/Azubi Africa BAP/LP5 - NLP/zindi_challenge/data/training_subset.csv",
                                "eval": "/content/drive/MyDrive/waid/Azubi Africa BAP/LP5 - NLP/zindi_challenge/data/eval_subset.csv"}, 
                    encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b80ee313c35136d9/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Define helper functions
## Function to transform labels
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2

    return {'labels': num}

## Function to tokenize data
def tokenize_data(example):
    return tokenizer(example['safe_text'], padding='max_length',truncation=True, max_length = 256)

In [ ]:
# Tokenize the tweets
dataset = data.map(tokenize_data, batched=True)

# Transform	labels and limit the columns
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/7999 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    "distilbert_covid_tweets_sentiment_analysis_model", 
    num_train_epochs=3,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    save_strategy="steps"
    )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Load the pretrained model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [ ]:
# Define evaluation metrics
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-20-4e99087ac86b>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
# Instantiate the training and evaluation sets
train_dataset = dataset["train"].shuffle(seed=24) 
eval_dataset = dataset["eval"].shuffle(seed=24)

In [ ]:
#converting training data to PyTorch tensors to speed up training and adding padding:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Instantiate the trainer
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset,compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7999
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3000
  Number of trainable parameters = 66955779


Step,Training Loss,Validation Loss,Accuracy
500,0.741400,0.684455,0.735000
1000,0.640000,0.635011,0.735000
1500,0.498600,0.619953,0.764500
2000,0.488000,0.608599,0.769000
2500,0.301600,0.788209,0.772000
3000,0.301100,0.780757,0.773000


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to distilbert_covid_tweets_sentiment_analysis_model/checkpoint-500
Configuration saved in distilbert_covid_tweets_sentiment_analysis_model/checkpoint-500/config.json
Model weights saved in distilbert_covid_tweets_sentiment_analysis_model/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to distilbert_covid_tweets_sentiment_analysis_model/checkpoint-1000
Configuration saved in distilbert_covid_tweets_sentiment_analysis_model/checkpoint-1000/config.json
Model weights saved in distilbert_covid_tweets_sentiment_analysis_model/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to distilbert_covid_tweets_sentiment_analysis_model/checkpoint-1500
Configuration saved in distilbert_covid_tweets_sentiment_analysis_model/checkpoint-1500/config.json
Model

TrainOutput(global_step=3000, training_loss=0.49513951110839843, metrics={'train_runtime': 669.76, 'train_samples_per_second': 35.829, 'train_steps_per_second': 4.479, 'total_flos': 1589438427646464.0, 'train_loss': 0.49513951110839843, 'epoch': 3.0})

In [ ]:
# Reinstantiate the trainer for evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Launch the final evaluation 
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6085986495018005,
 'eval_accuracy': 0.769,
 'eval_runtime': 16.6291,
 'eval_samples_per_second': 120.271,
 'eval_steps_per_second': 15.034}

In [ ]:
# Push model and tokenizer to HF Hub
model.push_to_hub("KwameOO/covid-tweet-sentiment-analyzer-distilbert")
tokenizer.push_to_hub("KwameOO/covid-tweet-sentiment-analyzer-distilbert")

Configuration saved in /tmp/tmpbi7dw_mj/config.json
Model weights saved in /tmp/tmpbi7dw_mj/pytorch_model.bin
Uploading the following files to KwameOO/covid-tweet-sentiment-analyzer-distilbert: config.json,pytorch_model.bin
tokenizer config file saved in /tmp/tmph_j7ll6r/tokenizer_config.json
Special tokens file saved in /tmp/tmph_j7ll6r/special_tokens_map.json
Uploading the following files to KwameOO/covid-tweet-sentiment-analyzer-distilbert: special_tokens_map.json,tokenizer_config.json,tokenizer.json,vocab.txt


CommitInfo(commit_url='https://huggingface.co/KwameOO/covid-tweet-sentiment-analyzer-distilbert/commit/e5f800832ca4622b606dd4691a0a7fc2693c4102', commit_message='Upload tokenizer', commit_description='', oid='e5f800832ca4622b606dd4691a0a7fc2693c4102', pr_url=None, pr_revision=None, pr_num=None)